### Домашнее задание к лекции "Основы веб-скрапинга"
Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список слов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по всем ключевым словам, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких ключевых слов из списка.

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [32]:
search_list_example = ['python','java','html']

In [45]:
def all_search_link(search_list):
    """
Функция выполняет парсинг первой старницы по поисковому запросу.
Возвращает все ссылки на статьи первой страницы поисковой выдачи.
Работает с habr.com
"""
    URL = f'https://habr.com/ru/search/'
    #Создадим список, куда будем записывать спарсенные ссылки
    fin_all_ref = []
    #Пройдемся циклом по всему списку ключевых слов для поиска
    for search_link in search_list:
        
        #Зададим общий справочник параметров для парсинга с указанной переменной search_link чтобы искать по списку ключевых слов
        params = {
            'q': search_link,
            'target_type': 'posts',
            'order': 'relevance'
                }
        headers = {}
        #Создадим запрос к искомому URL с указанными параметрами
        req = requests.get(URL, params)
        #Создадим объект бьютефул соуп, чтобы искать конкретные элементы с помощью библиотеки
        soup = BeautifulSoup(req.text, 'html.parser')
        #Ищем блок статей на старнице поисковой выдачи ()
        news_blocks = soup.find_all('div', class_='tm-article-snippet')
        #В найденном блоке ищем заголовки внутри них лежат ссылки
        articles_intro = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), news_blocks))
        #получаем ссылки из заголовков
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        #Ко всем ссылкам добавим обязательную часть начала сайта
        all_refs = list(map(lambda x: 'https://habr.com' + x, a_list))
        #добавим полученные парсингом ссылки в список
        fin_all_ref += all_refs
        #после поискового запроса сделаем паузу, чтобы не получить бан (в списке несколько ключевых слов)
        time.sleep(0.3)
    
    
    # создаем дф, в который будем записывать ссылки на статьи,дату статьи и  
    habr_article = pd.DataFrame()
    
    # собираем даты и заголовки статей из поисковой выдачи
    for link in set(fin_all_ref):
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
        #идем через if чтобы избежать ошибок связанных с разными структурами сайта
        #в итоговом датафрейме посмотрим на результат и есть ли пустые значения. Добавим условия для других вариантов структуры
        if soup.find('div', class_='tm-article-presenter__header'):
            date = pd.to_datetime(soup.find('div', class_='tm-article-presenter__header').find('time').get('datetime'), dayfirst=True).date()
        if soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1'): 
            title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
        #формируем дф из одной строки нужного нам вида
        row = {'date': date, 'link': link, 'title': title}
        #склеиваем построчно дф с данными нужного нам вида
        habr_article = pd.concat([habr_article, pd.DataFrame([row])])
        #удаляем дубликаты по заголовку (ссылки разные, статьи одни и те же)
        habr_article_clear = df_link_habr.drop_duplicates(subset = 'title')
    return habr_article_clear

In [51]:
#промежуточная проверка выдачи функции  
search_list_example = ['python','java','html']
df_link_habr = all_search_link(search_list_example)
df_link_habr

,date,link,title
0,2013-04-26,https://habr.com/ru/post/178081/,WebMarkupMin HTML Minifier – современный HTML-...
0,2012-12-26,https://habr.com/ru/post/163979/,Бенчмарк HTML парсеров
0,2008-12-21,https://habr.com/ru/post/47443/,HTML 5: пять вещей вызывающих особый интерес
0,2014-08-10,https://habr.com/ru/post/231845/,HTML-импорт — include для веба: часть 2
0,2022-01-20,https://habr.com/ru/company/sberbank/news/t/64...,Сбер проводит One Day Offer для Java-разработч...
0,2022-01-13,https://habr.com/ru/company/southbridge/news/t...,Открытый урок «Пишем Custom Prometheus Exporte...
0,2020-10-20,https://habr.com/ru/news/t/524284/,«1С: Предприятие» будет работать на российской...
0,2021-10-12,https://habr.com/ru/company/epam_systems/news/...,EPAM Anywhere ждёт Java-разработчиков на Hirin...
0,2021-09-27,https://habr.com/ru/company/southbridge/news/t...,«Разработчикам вход запрещен» или курс «Python...
0,2021-11-16,https://habr.com/ru/company/epam_systems/news/...,EPAM разработала бесплатный курс по программир...


### Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [62]:
def all_search_link(search_list, page_count):
    """
Функция выполняет парсинг первой старницы по поисковому запросу.
Возвращает все ссылки на статьи первой страницы поисковой выдачи.
Работает с habr.com

Параметры функции: 
1. search_list - параметр функции. На вход подаем список строк. Строки - поисковые запросы.
Как пример search_list_example = ['python','java','html']

2. page_count - кол-во страниц, которые парсим в поисковой выдаче по каждому ключевому слову. 
"""
    #Создадим список, куда будем записывать спарсенные ссылки
    fin_all_ref = []
   
    #Пройдемся циклом по всему списку ключевых слов для поиска
    for search_link in search_list:
        #зададим функцию x для переключения на новый url поиска
        #Обновляем счетчик страниц для следующего ключегового слова 
        x = 1
        #Обновляем счетчик страниц для следующего ключегового слова 
        page_number = page_count
        #добавим цикл для обработки нескольких номеров поисковых страниц
        while page_number != 0:
            #не нашел изменений в хедах запросов. Вместе с тем, меняются URL при загрузке новой страницы
            if x == 1: URL = 'https://habr.com/ru/search/'
            else: URL = f'https://habr.com/ru/search/page{x}/'
            x += 1
            page_number -= 1
            #Зададим общий справочник параметров для парсинга с указанной переменной search_link чтобы искать по списку ключевых слов
            params = {
                'q': search_link,
                'target_type': 'posts',
                'order': 'relevance'
                    }
            headers = {}
            #Создадим запрос к искомому URL с указанными параметрами
            req = requests.get(URL, params)
            #Добавим проверку на кол-во страниц (если мы захотим тех страниц, которых нет - получим бэд реквест)
            if req == False : break
            else:
                #Создадим объект бьютефул соуп, чтобы искать конкретные элементы с помощью библиотеки
                soup = BeautifulSoup(req.text, 'html.parser')
                
                #Добавим проверку на содержимое (если в ключевом слове дичь - то статус реквест будет 200, но страница пустой)
                if soup.find_all('div', class_='tm-article-snippet') == False : break
                else:
                    #Ищем блок статей на старнице поисковой выдачи ()
                    news_blocks = soup.find_all('div', class_='tm-article-snippet')
                    #В найденном блоке ищем заголовки внутри них лежат ссылки
                    articles_intro = list(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), news_blocks))
                    #получаем ссылки из заголовков
                    a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
                    #Ко всем ссылкам добавим обязательную часть начала сайта
                    all_refs = list(map(lambda x: 'https://habr.com' + x, a_list))
                    #добавим полученные парсингом ссылки в список
                    fin_all_ref += all_refs
                    #после поискового запроса сделаем паузу, чтобы не получить бан (в списке несколько ключевых слов)
                    time.sleep(0.3)
            
    
    
    # создаем дф, в который будем записывать ссылки на статьи,дату статьи и  
    habr_article = pd.DataFrame()
    
    # собираем даты и заголовки статей из поисковой выдачи
    for link in set(fin_all_ref):
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
        #идем через if чтобы избежать ошибок связанных с разными структурами сайта
        #в итоговом датафрейме посмотрим на результат и есть ли пустые значения. Добавим условия для других вариантов структуры
        if soup.find('div', class_='tm-article-presenter__header'):
            date = pd.to_datetime(soup.find('div', class_='tm-article-presenter__header').find('time').get('datetime'), dayfirst=True).date()
        if soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1'): 
            title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
        #формируем дф из одной строки нужного нам вида
        row = {'date': date, 'link': link, 'title': title}
        #склеиваем построчно дф с данными нужного нам вида
        habr_article = pd.concat([habr_article, pd.DataFrame([row])])
        #удаляем дубликаты по заголовку (ссылки разные, статьи одни и те же)
        habr_article_clear = df_link_habr.drop_duplicates(subset = 'title')
    return habr_article

In [63]:
page_count = 4
search_list_example = ['python','java','html']
df_link_habr_4 = all_search_link(search_list_example, page_count)
df_link_habr_4

,date,link,title
0,2019-09-06,https://habr.com/ru/post/466441/,Глючный код на Python: 10 самых распространенн...
0,2018-08-08,https://habr.com/ru/post/419637/,[Перевод] Как работает Graal — JIT-компилятор ...
0,2018-08-31,https://habr.com/ru/company/ruvds/blog/421741/,Почему человек из мира Java стал горячим сторо...
0,2014-10-28,https://habr.com/ru/post/241710/,Генерация HTML: удобнее чем хелперы и чистый HTML
0,2019-08-20,https://habr.com/ru/post/464405/,Python как предельный случай C++. Часть 2/2
...,...,...,...
0,2020-10-06,https://habr.com/ru/news/t/522224/,Приглашаем на осенний онлайн-практикум SimbirS...
0,2011-12-20,https://habr.com/ru/post/134744/,HTML.next или идеи для HTML6
0,2016-02-04,https://habr.com/ru/post/275729/,Руководство по HTML/CSS/JavaScript
0,2016-06-01,https://habr.com/ru/post/302432/,Используем HTML и WebBrowser control в качеств...
